In [1]:
import numpy as np

In [2]:
lines = []
with open("Automated_Traffic_Volume_Counts_20241127.csv", "r") as f:
    lines = f.readlines()[1:]

In [3]:
pos2id = {}

for line in lines:
    xs = line.split(",")
    pos = xs[9]
    if not pos in pos2id:
        num = len(pos2id.keys())
        pos2id[pos] = num

In [4]:
import pyproj

s_crs = pyproj.CRS.from_string("EPSG:2263")
t_crs = pyproj.CRS.from_string("EPSG:4326")

x, y = 997407.0998491726, 208620.92612708386

transformer = pyproj.Transformer.from_crs(s_crs, t_crs, always_xy=True)
t_p = transformer.transform(x, y)
print(t_p)

(-73.95252157233777, 40.7392830527809)


In [7]:
minn, maxx = 200001, 202405

In [8]:
N = 3598
time2v = {}
for line in lines:
    line = line.split(",")
    tmp = int(line[2]) * 100 + int(line[3])
    if not tmp in time2v:
        time2v[tmp] = np.zeros((N, 1), dtype=int)
    idd = pos2id[line[9]]
    time2v[tmp][idd,0] += int(line[7])

In [9]:
datas = []
timestp = list(time2v.keys())
timestp.sort()
for time_ in timestp:
    datas.append(time2v[time_].reshape(1, -1, 1))


In [12]:
m_data = np.vstack(datas)

In [13]:
m_data.shape

(197, 3598, 1)

In [14]:
np.savez("NYTV_month.npz", data=m_data)

In [7]:
import numpy as np

N = 3598
A = np.zeros((N, N), dtype=np.int8)
with open("NYTV_wkt.csv", "r") as f:
    lines = f.readlines()[1:]
    for line in lines:
        a = line.strip().split(",")
        i, j = int(a[0]), int(a[1])
        A[i,j]=A[j,i]=1



In [8]:
N = 3598
topk_nodes = [1510, 2147, 1589, 2958, 1520,  888,  889, 2186,  527, 3358]
nodes = set()
for nd in topk_nodes:
    nodes.add(nd)

while True:
    if len(nodes) >= 500:
        break
    lst = []
    for node in nodes:
        for i in range(N):
            if A[i,node]:
                lst.append(i)
    for i in lst:
        nodes.add(i)

len(nodes)

507

In [9]:
# 重新映射
new_N = len(nodes)
new_id = {}
num = 0
for node in nodes:
    new_id[node] = num
    num += 1

In [10]:
old_id = {}
for k, v in new_id.items():
    old_id[v] = k

In [21]:
new_data = np.zeros((197, new_N, 1), dtype=np.uint16)
old_id = [0] * new_N
for k, v in new_id.items():
    old_id[v] = k
for i in range(new_N):
    new_data[:,i,0] = m_data[:,old_id[i],0]

In [22]:
new_data.shape

(197, 507, 1)

In [23]:
np.savez("NYTV_month_mini.npz", data=new_data)

In [11]:
predict_month = np.load("predict_month_mini.npz")["data"]
predict_month.shape

(12, 507, 1)

In [12]:
id2pos = [0] * N
for k, v in pos2id.items():
    id2pos[v] = k

In [14]:
times = [("2024", "06"), ("2024", "07"), ("2024", "08"), ("2024", "09"), ("2024", "10"), ("2024", "11"), ("2024", "12"), ("2025", "01"), ("2024", "02"), ("2024", "03"), ("2024", "04"), ("2025", "05")]
year = 2024
month = 6
day = 11
def id2jingwei(idd):
    pos = id2pos[idd]
    pos = pos[7:-1]
    pos = pos.split(" ")
    return transformer.transform(float(pos[0]), float(pos[1]))

with open("predict_month.csv", "x") as f:
    f.write("lat,lon,year,month,day,hour,total_volume\n")
    for i in range(507):
        lon, lat = id2jingwei(old_id[i])
        for j, t in enumerate(times):
            f.write("%f,%f,%s,%s,1,0,%f\n" % (lat, lon, t[0], t[1], float(max(0, predict_month[j,i,0]))))

In [21]:
a_lines = []
with open("processed_data_with_hour.csv", "r") as f:
    a_lines = f.readlines()[1:]

lonlats = []
for i in range(507):
    lon, lat = id2jingwei(old_id[i])
    lonlats.append((lon, lat))

with open("processed_data_with_hour_mini.csv", "x") as f:
    f.write("lat,lon,year,month,day,hour,street,total_volume,avg_volume\n")
    for line in a_lines:
        a_lon, a_lat = float(line.split(",")[1]), float(line.split(",")[0])
        flag = False
        for t in lonlats:
            lon, lat = t
            if a_lon - lon <= 1e-5 and a_lat - lat <= 1e-5:
                flag = True
                break
        if flag:
            f.write(line.strip() + "\n")
        
